# Dataset
[Plant Village](https://data.mendeley.com/datasets/tywbtsjrjv/1)


# Description

In this data-set, 39 different classes of plant leaf and background images are available. The data-set containing 61,486 images. We used six different augmentation techniques for increasing the data-set size. The techniques are image flipping, Gamma correction, noise injection, PCA color augmentation, rotation, and Scaling.

The classes are:
1. Apple_scab
2. Apple_black_rot
3. Apple_cedar_apple_rust
4. Apple_healthy
5. Background_without_leaves
6. Blueberry_healthy
7. Cherry_powdery_mildew
8. Cherry_healthy
9. Corn_gray_leaf_spot
10. Corn_common_rust
11. Corn_northern_leaf_blight
12. Corn_healthy
13. Grape_black_rot
14. Grape_black_measles
15. Grape_leaf_blight
16. Grape_healthy
17. Orange_haunglongbing
18. Peach_bacterial_spot
19. Peach_healthy
20. Pepper_bacterial_spot
21. Pepper_healthy
22. Potato_early_blight
23. Potato_healthy
24. Potato_late_blight
25. Raspberry_healthy
26. Soybean_healthy
27. Squash_powdery_mildew
28. Strawberry_healthy
29. Strawberry_leaf_scorch
30. Tomato_bacterial_spot
31. Tomato_early_blight
32. Tomato_healthy
33. Tomato_late_blight
34. Tomato_leaf_mold
35. Tomato_septoria_leaf_spot
36. Tomato_spider_mites_two-spotted_spider_mite
37. Tomato_target_spot
38. Tomato_mosaic_virus
39. Tomato_yellow_leaf_curl_virus

In [1]:
%matplotlib inline

#
# import utilitaires
#
import cv2
import pandas as pd

from dataset_helpers import dataset_load_config, dataset_load
from jupyter_helpers import display_html

In [3]:
#
# installation dataset PlantVillage
#
dataset_config = dataset_load_config("PlantVillage_Config.json")
if dataset_config is None:
    display_html(f"<b>Invalid dataset config</b>")

dataset = dataset_load(dataset_config)
if dataset is None:
    display_html(f"<b>Invalid dataset</b>")

display(dataset)

,species,disease,image_paths
0,Apple,Apple_scab,dataset\Apple___Apple_scab\image (1).JPG
1,Apple,Apple_scab,dataset\Apple___Apple_scab\image (10).JPG
2,Apple,Apple_scab,dataset\Apple___Apple_scab\image (100).JPG
3,Apple,Apple_scab,dataset\Apple___Apple_scab\image (1000).JPG
4,Apple,Apple_scab,dataset\Apple___Apple_scab\image (101).JPG
...,...,...,...
60338,Tomato,Tomato_Yellow_Leaf_Curl_Virus,dataset\Tomato___Tomato_Yellow_Leaf_Curl_Virus...
60339,Tomato,Tomato_Yellow_Leaf_Curl_Virus,dataset\Tomato___Tomato_Yellow_Leaf_Curl_Virus...
60340,Tomato,Tomato_Yellow_Leaf_Curl_Virus,dataset\Tomato___Tomato_Yellow_Leaf_Curl_Virus...
60341,Tomato,Tomato_Yellow_Leaf_Curl_Virus,dataset\Tomato___Tomato_Yellow_Leaf_Curl_Virus...
